In [ ]:
import sys
print(sys.version)

In [ ]:
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
%matplotlib inline

In [ ]:
sys.path.append("../code/")

In [ ]:
from pca import GenePCA, plot_pca_results, plot_color_by_week, plot_color_by_category

https://github.com/JanetMatsen/elvizAnalysis/blob/dc568af7ed589410bf9612ba5ec7ccf257e87d1e/elviz_pca.py

http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
sample_info = pd.read_csv('../data/sample_info.tsv', sep='\t')

In [ ]:
sample_info.head()

In [ ]:
sample_info['O2 & replicate'] = sample_info.oxy.str.cat(sample_info.rep.astype(str), sep=' ')

In [ ]:
all_X = pd.read_csv('../data/raw_data/3_summary_rpkm_byGeneProuct.xls', sep = '\t')
all_X.head(2)

In [ ]:
all_X.shape

In [ ]:
# http://stackoverflow.com/questions/31663426/python-pandas-drop-rows-from-data-frame-on-string-match-from-list
# to_drop = ['Clerk', 'Bagger']
# df[~df['title'].isin(to_drop)]
# df[~df['title'].str.contains('|'.join(to_drop))]
drop_names = ['unique', 'unknown']

all_X = all_X[~all_X['product'].str.contains('|'.join(drop_names))]
all_X.shape

In [ ]:
# Hack 12/7: remove the genes with unknown function.  

In [ ]:
all_X.shape

In [ ]:
X = all_X.drop(['product'], axis=1).T
X.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
# X : numpy array of shape [n_samples, n_features]
#print(X.to_array()[0:5, 0:5])
ss = StandardScaler()
X_ss = ss.fit_transform(X)
print(X_ss[0:5, 0:5])

In [ ]:
X.reset_index(inplace=True)
X = X.rename(columns={'index':'sample'})
SAMPLE_NAMES = X['sample']

In [ ]:
X_ss.shape

In [ ]:
SAMPLE_NAMES.shape

In [ ]:
sample_info.shape

In [ ]:
model = GenePCA(raw_x = X_ss, sample_names=SAMPLE_NAMES, 
                gene_names=all_X['product'], sample_info=sample_info)

In [ ]:
model.fit()

In [ ]:
f1 = plot_color_by_week(plot_data=model.plot_data, variances=model.explained_variance,
                       plot_pca_kwargs={'facet_row':True, 'uniform_axes':True})

In [ ]:
f = plot_color_by_week(plot_data=model.plot_data, variances=model.explained_variance,
                       plot_pca_kwargs={'facet_row':False, 'uniform_axes':False, 
                                        'subplot_size':6})

In [ ]:
f.savefig("../figures/161207_PCA_color_by_week.pdf")
f.savefig("../figures/161207_PCA_color_by_week.png")

In [ ]:
model.component_features.head(2)

In [ ]:
model.component_features.to_csv("../data/normalized_pca_eigenvectors.csv")

In [ ]:
model.component_features.head(10)

In [ ]:
plot_color_by_category(plot_data=model.plot_data, variances=model.explained_variance,
                       categories='O2 & replicate', n_categories=8,
                       plot_pca_kwargs={'facet_row':False, 'uniform_axes':False, 
                                        'subplot_size':6,
                                       'filename':'../figures/161207_PCA--color_by_O2_replicate', 
                                        'savefig':True})

### TODO:  sort only on first eigenvector's weights.  

More important than all the eigenvectors.
Also, are the eigenvectors in order of importance?  (should be yes)

In [ ]:
eigenvalue_1_genes = model.component_features[[0, 'product']]
eigenvalue_1_genes.head(10)

In [ ]:
eigenvalue_1_genes.sort_values(by=0, ascending=False, inplace=True)
eigenvalue_1_genes.head()

In [ ]:
# Why are the weights mostly the same and large?  (Artifact of normalization?)
eigenvalue_1_genes.plot.hist(figsize=(4,2.5))